In [26]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [27]:
import sys

sys.path.append("../")

from src.eda.loader import DataLoader

loader = DataLoader("../data/raw/complaints.csv")

df = loader.get_data()
df.head()

c:\Users\hp\Desktop\Projects\ML\KAIM\week-6\fincomplain-ragbot\notebooks\..\src\eda\loader.py:13: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.filepath)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,Experian Information Solutions Inc.,FL,32092,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195687
1,2025-06-20,Debt collection,Telecommunications debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company can't verify or dispute the facts in t...,"Eastern Account Systems of Connecticut, Inc.",FL,342XX,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14195688
2,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195689
3,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195690
4,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,Experian Information Solutions Inc.,IL,60628,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195692


### EDA

In [29]:
df.info()
df_saved = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9609797 entries, 0 to 9609796
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(17)
memory usage: 1.3+

In [30]:
df.isnull().sum().sort_values(ascending=False)

Tags                            8981029
Consumer disputed?              8841498
Consumer complaint narrative    6629041
Company public response         4770207
Consumer consent provided?      1649561
Sub-issue                        839522
Sub-product                      235295
State                             54516
ZIP code                          30228
Company response to consumer         20
Issue                                 6
Date received                         0
Product                               0
Company                               0
Date sent to company                  0
Submitted via                         0
Timely response?                      0
Complaint ID                          0
dtype: int64

In [31]:
df['Product'].value_counts()

Product
Credit reporting or other personal consumer reports                             4834855
Credit reporting, credit repair services, or other personal consumer reports    2163857
Debt collection                                                                  799197
Mortgage                                                                         422254
Checking or savings account                                                      291178
Credit card                                                                      226686
Credit card or prepaid card                                                      206369
Money transfer, virtual currency, or money service                               145066
Credit reporting                                                                 140429
Student loan                                                                     109717
Bank account or service                                                           86205
Vehicle loan or lease   

### Preprocessing

#### Product filtering

In [32]:
def map_product(product: str) -> str:
    product = product.lower()

    # CREDIT CARD
    if product in [
        "credit card",
        "credit card or prepaid card",
        "prepaid card"
    ]:
        return "Credit Card"

    # PERSONAL LOAN
    elif product in [
        "consumer loan",
        "payday loan, title loan, or personal loan",
        "payday loan, title loan, personal loan, or advance loan",
        "payday loan"
    ]:
        return "Personal Loan"

    # BNPL
    elif "buy now" in product:
        return "BNPL"

    # SAVINGS ACCOUNT
    elif product in [
        "checking or savings account",
        "bank account or service"
    ]:
        return "Savings Account"

    # MONEY TRANSFERS
    elif product in [
        "money transfers",
        "money transfer, virtual currency, or money service"
    ]:
        return "Money Transfers"

    # All others: discard
    else:
        return None


In [36]:
df = df_saved.copy()

In [37]:
df["Product"] = df["Product"].apply(map_product)
df = df[df["Product"].notna()]

In [38]:
df["Product"].value_counts()


Product
Credit Card        448335
Savings Account    377383
Money Transfers    150420
Personal Loan       84270
Name: count, dtype: int64

#### Consumer compliant narrative

In [39]:
count = df['Consumer complaint narrative'].dropna().str.split().str.len()

In [40]:
print(count.min())
print(count.max())
print(count.std())
print(count.median())
print(count.mean())

1
6469
224.97469644484403
137.0
205.14300633144674


In [47]:
import re

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"i am writing to (file|submit) a complaint", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s\.\?]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [42]:
df.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
334,2025-06-20,Credit Card,General-purpose credit card or charge card,Problem with a company's investigation into an...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",TX,75104,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14196294
351,2025-06-07,Savings Account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,WELLS FARGO & COMPANY,TX,75034,NaN,NaN,Referral,2025-06-09,In progress,Yes,NaN,13984551
437,2025-06-07,Savings Account,Checking account,Closing an account,Company closed your account,NaN,NaN,WELLS FARGO & COMPANY,NV,89115,NaN,NaN,Web,2025-06-07,In progress,Yes,NaN,13956296
595,2025-06-20,Credit Card,General-purpose credit card or charge card,Problem with a company's investigation into an...,Was not notified of investigation status or re...,NaN,NaN,Experian Information Solutions Inc.,CA,92553,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14196854
661,2025-06-20,Credit Card,General-purpose credit card or charge card,Problem with a company's investigation into an...,Was not notified of investigation status or re...,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,77346,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14196870
687,2025-06-20,Credit Card,General-purpose credit card or charge card,Problem with a company's investigation into an...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",TX,77346,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14196880
795,2025-06-20,Credit Card,General-purpose credit card or charge card,Incorrect information on your report,Account status incorrect,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",LA,71108,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14197003
808,2025-06-20,Money Transfers,Virtual currency,Other service problem,NaN,NaN,NaN,"Block, Inc.",GA,30213,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14197060
1025,2025-06-20,Savings Account,Checking account,Closing an account,Funds not received from closed account,NaN,NaN,TransferWise Ltd,DE,19901,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14197335
1031,2025-06-20,Credit Card,General-purpose credit card or charge card,Getting a credit card,Card opened without my consent or knowledge,NaN,NaN,"EQUIFAX, INC.",TX,75065,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14197343


In [43]:

df = df[df['Consumer complaint narrative'].notna()]

In [44]:
df.dtypes

Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Date sent to company            object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
dtype: object

In [46]:
df['Consumer complaint narrative'].isnull().sum()

np.int64(0)

In [48]:
df = df[df['Consumer complaint narrative'].str.split().str.len() >= 1]
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
12237,2025-06-13,Credit Card,Store credit card,Getting a credit card,Card opened without my consent or knowledge,a xxxx xxxx card was opened under my name by a...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,78230,Servicemember,Consent provided,Web,2025-06-13,Closed with non-monetary relief,Yes,NaN,14069121
12532,2025-06-13,Savings Account,Checking account,Managing an account,Deposits and withdrawals,i made the mistake of using my wellsfargo debi...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,ID,83815,NaN,Consent provided,Web,2025-06-13,Closed with explanation,Yes,NaN,14061897
13280,2025-06-12,Credit Card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,dear cfpb i have a secured credit card with ci...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,11220,NaN,Consent provided,Web,2025-06-13,Closed with monetary relief,Yes,NaN,14047085
13506,2025-06-12,Credit Card,General-purpose credit card or charge card,Incorrect information on your report,Account information incorrect,i have a citi rewards cards. the credit balanc...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",IL,60067,NaN,Consent provided,Web,2025-06-12,Closed with explanation,Yes,NaN,14040217
13955,2025-06-09,Credit Card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,b i am writing to dispute the following charge...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,78413,Older American,Consent provided,Web,2025-06-09,Closed with monetary relief,Yes,NaN,13968411


In [53]:
df["Product"].value_counts()

Product
Credit Card        197126
Savings Account    155204
Money Transfers     98685
Personal Loan       37341
Name: count, dtype: int64

In [52]:
df.shape

(488356, 18)

In [51]:
df.to_csv("../data/processed/filtered_compliants.csv", index=False)

**Exploratory Analysis Summary**

After conducting EDA on the CFPB complaint dataset and performing targeted preprocessing, the following key insights emerged:

*   The raw dataset contained over 8.9 million complaints, but more than **6.6 million** were missing usable narrative text.
    
*   We focused on **five relevant product categories**, but only four had sufficient volume:**Credit Card, Personal Loan, Savings Account, and Money Transfers.**
    
*   Using an if-else mapping function, we normalized product names to ensure consistency across categories.
    
    *   Credit Card 197,126 
    *   Savings Account 155,204 
    *   Money Transfers 98,685 
    *   Personal Loan 37,341
    
*   Text cleaning included removing boilerplate phrases, converting to lowercase, and normalizing whitespace and punctuation.
    
*   A wide variance in narrative length confirmed the need for chunking before semantic search.